# AMMap Tool for Additive Manufacturing Design, Alloy Discovery, and Path Planning

--> ***(Please see https://github.com/amkrajewski/nimplex for setting up and utilizing nimplex)***

--> You will additionally need `pathfinding` libraries to run part of this exercise. If you are running this in Codespaces, it has been pre-installed for you.

**This notebook is the current method of utilizing AMMap and through minor alterations can be changed to run AMMap on any system from an Input YAML file**

**In this notebook, we will demonstrate how effortless it is to dramatically speed up the exploration of feasible compositional spaces in high dimensional spaces through employing `nimplex`'s graph representations that abstract the underlying problem and dimensionality.**

In [1]:
# Import nimplex and other necessary packages
import nimplex
import numpy as np
from utils import stitching
from IPython.display import clear_output
from itertools import combinations
import pandas as pd
from pycalphad import Database
import os
import importlib
import json
import igraph as ig
import plotly.graph_objs as go
import yaml
import re

In [2]:
# Set the seed for reproducibility
import random
random.seed(123)

In [3]:
# Load the YAML file
yaml_file = 'publication_input.yaml'
with open(yaml_file, 'r') as file:
    yaml_content = yaml.safe_load(file)


In [4]:
# Create thermodynamic callables
!python ammap/callableBuilders/construct_callables.py {yaml_file}

Equilibrium callable constructed: ammap/callables/multi_system_equilibrium_and_scheil/equilibrium_callable_CrTiV_ed4c332b.py
Scheil callable constructed: ammap/callables/multi_system_equilibrium_and_scheil/scheil_callable_CrTiV_ed4c332b.py
Equilibrium callable constructed: ammap/callables/multi_system_equilibrium_and_scheil/equilibrium_callable_NiCrV_b7aba9ab.py
Scheil callable constructed: ammap/callables/multi_system_equilibrium_and_scheil/scheil_callable_NiCrV_b7aba9ab.py
Equilibrium callable constructed: ammap/callables/multi_system_equilibrium_and_scheil/equilibrium_callable_CrFeV_ab1edb07.py
Scheil callable constructed: ammap/callables/multi_system_equilibrium_and_scheil/scheil_callable_CrFeV_ab1edb07.py
Equilibrium callable constructed: ammap/callables/multi_system_equilibrium_and_scheil/equilibrium_callable_NiCrFe_f434a6d9.py
Scheil callable constructed: ammap/callables/multi_system_equilibrium_and_scheil/scheil_callable_NiCrFe_f434a6d9.py
Equilibrium callable constructed: amma

In [5]:
for key in yaml_content.keys():
    print(key)

name
nDivisionsPerDimension
elementalSpaces
designSpaces
constraints
pathPlan


In [6]:
print(yaml_content.keys())
print(type(yaml_content['elementalSpaces']))
print(type(yaml_content['elementalSpaces'][0]))
print(type(yaml_content['constraints']))
print(type(yaml_content['constraints'][0]))
print(yaml_content['constraints'][0].keys())


dict_keys(['name', 'nDivisionsPerDimension', 'elementalSpaces', 'designSpaces', 'constraints', 'pathPlan'])
<class 'list'>
<class 'dict'>
<class 'list'>
<class 'dict'>
dict_keys(['type', 'temperature', 'pressure', 'feasiblePhases'])


In [7]:
#Reads all elements from the elementalSpaces (base elements)
elementalSpaceComponents = sorted(list({elem for element in yaml_content['elementalSpaces'] for elem in element['elements']}))
print(elementalSpaceComponents)

['Cr', 'Fe', 'Ni', 'Ti', 'V']


In [14]:
print("Possible entry options in design_spaces:")
for entry in design_spaces:
    print(list(entry.keys()))

Possible entry options in design_spaces:
['name', 'elementalSpace', 'components']
['stitch', 'targetDesignSpace']
['stitch', 'targetDesignSpace']
['stitch', 'targetDesignSpace']
['name', 'elementalSpace', 'components']
['name', 'elementalSpace', 'components']
['name', 'elementalSpace', 'components']
['name', 'elementalSpace', 'components']
['name', 'elementalSpace', 'components']
['name', 'elementalSpace', 'components']
['name', 'elementalSpace', 'components']
['name', 'elementalSpace', 'components']
['name', 'elementalSpace', 'components']


In [16]:
#Make this come from YAML (designSpaces)
design_spaces = yaml_content['designSpaces']
# Combine all the names of each entry in designSpaces
attainableSpaceComponents = []
for entry in design_spaces:
    if 'name' in entry:
        components = re.findall(r'[A-Z][a-z]*', entry['name'])
        attainableSpaceComponents.extend(components)

# Remove duplicates and sort the list
attainableSpaceComponents = sorted(set(attainableSpaceComponents))


print(attainableSpaceComponents)


['Cr', 'Fe', 'Ni', 'Ti', 'V']


In [17]:
import yaml
import re

# Load the YAML file
with open('example_input.yaml', 'r') as file:
    yaml_content = yaml.safe_load(file)

# Extract the designSpaces key
design_spaces = yaml_content['designSpaces']

# Combine all the names of each entry in designSpaces
attainableSpaceComponents = []
for entry in design_spaces:
    components = re.findall(r'[A-Z][a-z]*', entry['name'])
    attainableSpaceComponents.extend(components)

# Remove duplicates and sort the list
attainableSpaceComponents = sorted(set(attainableSpaceComponents))

# Print the results for verification
print("Attainable Space Components:")
print(attainableSpaceComponents)

Attainable Space Components:
['Cr', 'Ni', 'Ss', 'Ti', 'V']


In [18]:
import yaml
import re

# Load the YAML file
with open('example_input.yaml', 'r') as file:
    yaml_content = yaml.safe_load(file)

# Extract the designSpaces key
design_spaces = yaml_content['designSpaces']

# Combine all the names of each entry in designSpaces
attainableSpaceComponents = []
for entry in design_spaces:
    components = re.findall(r'[A-Z][a-z]*', entry['name'])
    attainableSpaceComponents.extend(components)

# Remove duplicates and sort the list
attainableSpaceComponents = sorted(set(attainableSpaceComponents))

# Print the results for verification
print("Attainable Space Components:")
print(attainableSpaceComponents)

Attainable Space Components:
['Cr', 'Ni', 'Ss', 'Ti', 'V']


In [19]:
#Make this come from YAML (designSpaces)
#Reads the desired components for the design space that may not be constrained to base elements
attainableSpaceComponents = ["Cr", "Fe", "Ni", "Ti", "V"]
attainableSpaceComponentPositions = [
    [1,0,0,0,0],
    [0,1,0,0,0],
    [0,0,1,0,0],
    [0,0,0,1,0],
    [0,0,0,0,1]
]    

In [20]:
ternaries = list(combinations(attainableSpaceComponents, 3))
ternaries_CompPos = list(combinations(attainableSpaceComponentPositions, 3))
ndiv = yaml_content['nDivisionsPerDimension']
gridAtt, nList = nimplex.simplex_graph_py(3, ndiv)

for tern, terncp in zip(ternaries, ternaries_CompPos):
    print(f"{str(tern):<40} -> {terncp}")

('Cr', 'Fe', 'Ni')                       -> ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0])
('Cr', 'Fe', 'Ti')                       -> ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0])
('Cr', 'Fe', 'V')                        -> ([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1])
('Cr', 'Ni', 'Ti')                       -> ([1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0])
('Cr', 'Ni', 'V')                        -> ([1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1])
('Cr', 'Ti', 'V')                        -> ([1, 0, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1])
('Fe', 'Ni', 'Ti')                       -> ([0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0])
('Fe', 'Ni', 'V')                        -> ([0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1])
('Fe', 'Ti', 'V')                        -> ([0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1])
('Ni', 'Ti', 'V')                        -> ([0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1])


In [23]:
print(terncp)

([0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1])


In [24]:
# Edges list for graph plotting and path finding purposes
edges = []
# Connectivity list within each subsystem
graphN = [[] for i in range(len(gridAtt * len(ternaries)))]
# Connectivity list between subsystems
graphNS = [[] for i in range(len(graphN))]
compositions = []
compositions_with_id = []  # List to store compositions with their identifiers
ternaries_with_id = []  # New list to store ternaries_CompPos with their identifiers

# Iterate over ternaries
for i, terncp in enumerate(ternaries_CompPos):
    ternaries_with_id.append((terncp, i))  # Add terncp and its id to the new list
    
    offset = i*len(gridAtt)
    for j in range(len(gridAtt)):
        for n in nList[j]:
            edges.append((j+offset,n+offset))
            graphN[j+offset].append(n+offset)
    print(terncp)
    gridAttTemp, gridElTemp = nimplex.embeddedpair_simplex_grid_fractional_py(terncp, ndiv)
    compositions += gridElTemp
    
    # Attach identifier to each composition
    compositions_with_id.extend([(comp, i) for comp in gridElTemp])

([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0])
([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 1, 0])
([1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1])
([1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0])
([1, 0, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1])
([1, 0, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1])
([0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0])
([0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 0, 1])
([0, 1, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1])
([0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1])


In [22]:
mapping = {}
for ternary, id in ternaries_with_id:
    composition_key = ''.join(elementalSpaceComponents[i] for i in range(len(elementalSpaceComponents)) if any(ternary[j][i] for j in range(len(ternary))))
    individual_elements = [elementalSpaceComponents[i] for i in range(len(elementalSpaceComponents)) if any(ternary[j][i] for j in range(len(ternary)))]
    mapping[composition_key] = {
        'id': id,
        'elements': individual_elements
    }

# Print the mapping to see the result
for key, value in mapping.items():
    print(f"Combination: {key}, ID: {value['id']}, Elements: {value['elements']}")

Combination: CrFeNi, ID: 0, Elements: ['Cr', 'Fe', 'Ni']
Combination: CrFeTi, ID: 1, Elements: ['Cr', 'Fe', 'Ti']
Combination: CrFeV, ID: 2, Elements: ['Cr', 'Fe', 'V']
Combination: CrNiTi, ID: 3, Elements: ['Cr', 'Ni', 'Ti']
Combination: CrNiV, ID: 4, Elements: ['Cr', 'Ni', 'V']
Combination: CrTiV, ID: 5, Elements: ['Cr', 'Ti', 'V']
Combination: FeNiTi, ID: 6, Elements: ['Fe', 'Ni', 'Ti']
Combination: FeNiV, ID: 7, Elements: ['Fe', 'Ni', 'V']
Combination: FeTiV, ID: 8, Elements: ['Fe', 'Ti', 'V']
Combination: NiTiV, ID: 9, Elements: ['Ni', 'Ti', 'V']


In [ ]:
stitchingBinaries = {}

for i, combo1 in enumerate(ternaries):
    for j, combo2 in enumerate(ternaries[i+1:], start=i+1):
        common = set(combo1) & set(combo2)
        if len(common) == 2:
            overlap = tuple(sorted(common))
            if overlap not in stitchingBinaries:
                stitchingBinaries[overlap] = []
            stitchingBinaries[overlap].append((i, j))

for overlap, pairs in stitchingBinaries.items():
    print(f"{overlap}: occurs between ternary {pairs}")

In [ ]:
for stitchingBinary, ternaryPairList in stitchingBinaries.items():
    for ternaryPair in ternaryPairList:
        ternary1, ternary2 = ternaryPair[0], ternaryPair[1]
        stitching1 = stitching.findStitchingPoints_py(
            3, ndiv, 
            components=ternaries[ternary1],
            offset=ternary1*len(gridAtt)
            )["-".join(stitchingBinary)]
        stitching2 = stitching.findStitchingPoints_py(
            3, ndiv, 
            components=ternaries[ternary2],
            offset=ternary2*len(gridAtt)
            )["-".join(stitchingBinary)]
        print(f"Stitching {ternary1} and {ternary2} at {stitchingBinary} from {stitching1} to {stitching2}")
        for i, j in zip(stitching1, stitching2):
            #To
            edges.append((i, j))
            graphNS[i].append(j)
            #From
            edges.append((j, i))
            graphNS[j].append(i)

In [ ]:
db_files = [
    "ammap/databases/Co-Cr-Fe-Ni-V_choi2019.TDB",
    "ammap/databases/Cr-Fe-Ti_wang2017.tdb",
    "ammap/databases/Cr-Fe-Ni_miettinen1999.tdb",
    "ammap/databases/Cr-Ni-Ti_huang2018.tdb",
    "ammap/databases/Cr-Ti-V_ghosh2002.tdb",
    "ammap/databases/Fe-Ni-Ti_dekeyzer2009.tdb",
    "ammap/databases/Fe-Ni-V_zhao2014.tdb",
    "ammap/databases/Fe-Ti-V_guo2012.TDB",
    "ammap/databases/Ni-Ti-V_zou2018.tdb"
]

# Dictionary to store unique phases for each database
unique_phases = {}

# Iterate through each database file
for db_file in db_files:
    dbf = Database(db_file)
    phases = list(set(dbf.phases.keys()))
    unique_phases[db_file] = phases

# Print unique phases for each database
for db_file, phases in unique_phases.items():
    print(f"Unique phases for {db_file}: {phases}")

In [ ]:
phases = list(set(dbf.phases.keys()))
print(elementalSpaceComponents)
print(f'Loaded TDB file with phases considered: {phases}')

In [ ]:
# Directory containing the equilibrium files
directory = "ammap/callables/multi_system_equilibrium_and_scheil"

# Get all files starting with "equilibrium"
equilibrium_files = [f for f in os.listdir(directory) if f.startswith("equilibrium") and f.endswith(".py")]

# Dictionary to store imported callables with unique names
equilibrium_callables = {}

# Import each equilibrium file and store the callable with a unique name
for file in equilibrium_files:
    module_name = file[:-3]  # Remove the .py extension
    module_path = f"ammap.callables.multi_system_equilibrium_and_scheil.{module_name}"
    module = importlib.import_module(module_path)
    callable_name = f"{module_name}"
    equilibrium_callables[callable_name] = getattr(module, "equilibrium_callable")

# Print the imported callables
for name, func in equilibrium_callables.items():
    print(f"Imported {name}: {func}")

In [ ]:
scheil_files = [f for f in os.listdir(directory) if f.startswith("scheil") and f.endswith(".py")]
scheil_callables = {}

for file in scheil_files:
    module_name = file[:-3]
    module_path = f"ammap.callables.multi_system_equilibrium_and_scheil.{module_name}"
    module = importlib.import_module(module_path)
    callable_name = f"{module_name}"
    scheil_callables[callable_name] = getattr(module, "scheil_callable")

for name, func in scheil_callables.items():
    print(f"Imported {name}: {func}")

In [ ]:
def process_filename(filename):
    parts = filename.split('_')
    if len(parts) >= 4:
        middle_part = parts[2]
        if middle_part in element_mapping:
            return middle_part, element_mapping[middle_part]
    return None, None

In [ ]:
def process_key(key):
    parts = key.split('_')
    if len(parts) >= 3:
        middle_part = parts[2]
        if middle_part in mapping:
            return middle_part, mapping[middle_part]['id'], mapping[middle_part]['elements']
        
        # If direct matching fails, try matching by elements
        middle_elements = set(middle_part[i:i+2] for i in range(0, len(middle_part), 2))
        for map_key, value in mapping.items():
            if set(value['elements']) == middle_elements:
                return middle_part, value['id'], value['elements']
    
    return None, None, None

# Process each key in the equilibrium_callables dictionary
id_to_callable = {}
for key in equilibrium_callables:
    middle_part, mapping_id, elements = process_key(key)
    if middle_part and mapping_id is not None:
        print(f"Key: {key}")
        print(f"The mapping number for {middle_part} is {mapping_id}")
        print(f"Elements: {elements}")
        print("---")
        id_to_callable[mapping_id] = key
    else:
        print(f"No mapping found for key: {key}")
        print("---")

sc_id_to_callable = {}
for key in scheil_callables:
    middle_part, mapping_id, elements = process_key(key)
    if middle_part and mapping_id is not None:
        print(f"Key: {key}")
        print(f"The mapping number for {middle_part} is {mapping_id}")
        print(f"Elements: {elements}")
        print("---")
        sc_id_to_callable[mapping_id] = key
    else:
        print(f"No mapping found for key: {key}")
        print("---")

In [ ]:
def reduce_compositions(compositions_with_id, mapping):
    element_order = elementalSpaceComponents
    system_comps_with_id = []
    reduced_compositions = []
    
    for composition, comp_id in compositions_with_id:
        relevant_entry = next((entry for entry in mapping.values() if entry['id'] == comp_id), None)
        if not relevant_entry:
            continue
        
        relevant_indices = [element_order.index(elem) for elem in relevant_entry['elements']]
        reduced_point = [composition[index] for index in relevant_indices]
        system_comps_with_id.append((reduced_point, comp_id))
        reduced_compositions.append(reduced_point)
    
    return system_comps_with_id, reduced_compositions

In [ ]:
system_comps_with_id, reduced_compositions = reduce_compositions(compositions_with_id, mapping)

In [ ]:
from tqdm.contrib.concurrent import process_map

In [ ]:
startingNodes = [0, 90, 20, 31] + random.sample(range(len(reduced_compositions)), 11)
print(f"Starting nodes: {startingNodes}")

for startingNode in startingNodes:
    print(f"Starting node: {reduced_compositions[startingNode]}")

In [ ]:
gridFeasible = [None]*len(reduced_compositions)
queue = startingNodes.copy()
explored = set()
calcCount = 0

In [ ]:
# Function to get the correct callable for a given composition
def get_callable(composition):
    for comp, id in compositions_with_id:
        if comp == composition:
            callable_name = id_to_callable.get(id)
            if callable_name is None:
                raise ValueError(f"No callable name found for ID {id}")
            if callable_name in globals():
                return globals()[callable_name]
            else:
                raise NameError(f"Function '{callable_name}' not found in global scope")
    raise ValueError(f"No callable found for composition {composition}")

def process_composition(elP):
    try:
        callable_func = get_callable(elP)
        return callable_func(elP)
    except Exception as e:
        print(f"Error processing composition {elP}: {str(e)}")
        return None

In [ ]:
# Function to get the correct callable for a given composition
def get_sc_callable(composition):
    for comp, id in compositions_with_id:
        if comp == composition:
            callable_name = sc_id_to_callable.get(id)
            if callable_name is None:
                raise ValueError(f"No callable name found for ID {id}")
            if callable_name in globals():
                return globals()[callable_name]
            else:
                raise NameError(f"Function '{callable_name}' not found in global scope")
    raise ValueError(f"No callable found for composition {composition}")

def process_sc_composition(elP):
    try:
        callable_func = get_sc_callable(elP)
        return callable_func(elP)
    except Exception as e:
        print(f"Error processing composition {elP}: {str(e)}")
        return None

In [ ]:
from functools import partial

def get_equilibrium_callable(composition, id_to_callable, equilibrium_callables):
    composition_id = composition[1]  # Get the ID from the composition tuple
    #print(f"Composition ID: {composition_id}")
    callable_name = id_to_callable.get(composition_id)
    if callable_name is None:
        raise ValueError(f"No callable found for composition ID {composition_id}")
    callable_func = equilibrium_callables.get(callable_name)
    if callable_func is None:
        raise ValueError(f"No callable function found for name {callable_name}")
    return callable_func

def apply_equilibrium_callable(callable_and_position):
    callable_func, position = callable_and_position
    return callable_func(position)

In [ ]:
def get_scheil_callable(composition, sc_id_to_callable, scheil_callables):
    composition_id = composition[1]  # Get the ID from the composition tuple
    #print(f"Composition ID: {composition_id}")
    callable_name = sc_id_to_callable.get(composition_id)
    if callable_name is None:
        raise ValueError(f"No callable found for composition ID {composition_id}")
    callable_func = scheil_callables.get(callable_name)
    if callable_func is None:
        raise ValueError(f"No callable function found for name {callable_name}")
    return callable_func

def apply_scheil_callable(callable_and_position):
    callable_func, position = callable_and_position
    return callable_func(position)

In [ ]:
# Initialize a list to store the results
results_list = []

while len(queue) > 0:
    print(f"Queue: {queue}")
    # Calculate feasibilities of the current queue
    elPositions = [reduced_compositions[i] for i in queue]
    print(elPositions)
    # Create a list of equilibrium callables for each composition
    equilibrium_callables_list = [get_equilibrium_callable(system_comps_with_id[i], id_to_callable, equilibrium_callables) for i in queue]
    
    if len(queue) > 3:
        results = process_map(apply_equilibrium_callable, zip(equilibrium_callables_list, elPositions), max_workers=4)
    else:
        results = [ec(elP) for ec, elP in zip(equilibrium_callables_list, elPositions)]
    
    # Extract only the 'Phases' component from the results
    phases = [result['Phases'] for result in results]
    
    #feasibilities = [len(set(p) & set(['FCC_A1', 'BCC_A2', 'HCP_A3', 'B2_BCC','A2_FCC','L12_FCC','BCC2', 'A1', 'A2', 'A3', 'FCC4'])) == 0 and p != [] for p in phases]
    feasibilities = [set(p).issubset(set(['FCC_A1', 'BCC_A2', 'HCP_A3', 'B2_BCC', 'A2_FCC', 'L12_FCC', 'BCC2', 'A1', 'A2', 'A3', 'FCC4'])) and p != [] for p in phases]

    calcCount += len(feasibilities)
    explored = explored.union(queue)

    # Save the current step result and elPositions
    results_list.append({
        'queue': queue,
        'elPositions': elPositions,
        'results': results
    })

    # Create next queue based on neighbors of feasible points
    nextQueue = set()
    #nextQueuePlusEquivalent = set()
    for f, i in zip(feasibilities, queue):
        # Explored point
        gridFeasible[i] = f

        # # And equivalent explored points based on system stitching
        # explored = explored.union(graphNS[i])
        # for eq in graphNS[i]:
        #     gridFeasible[eq] = f

        # Expand to neighbors of the point and equivalent points (only if the node has been feasible)
        if f:
            # Node neighbors in the same subsystem
            for n in graphN[i]:
                if n not in explored:# and n not in nextQueuePlusEquivalent:
                    nextQueue.add(n)
                    #nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])
            # Equivalent nodes neighbors in other subsystems
            for eq in graphNS[i]:
                #for n in graphN[eq]:
                if eq not in explored:# and n not in nextQueuePlusEquivalent:
                    nextQueue.add(eq)
                    #nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])

    print(f"Calculations done: {calcCount:<5} | Explored points: {len(explored):<5}")
    queue = list(nextQueue)

# Write the results to a JSON file
with open('results.json', 'w') as f:
    json.dump(results_list, f, indent=4)

In [ ]:
# Load the results from the JSON file
with open('/ocean/projects/dmr190011p/arichte1/github_repo/AMMap/results.json', 'r') as f:
    data = json.load(f)

# Initialize a dictionary to store the merged results
merged_results = {}

# Iterate through each step in the data
for step in data:
    queue = step['queue']
    results = step['results']
    
    # Merge the queue numbers with their associated result values
    for q, result in zip(queue, results):
        merged_results[q] = result

# Convert the merged results dictionary to a list of dictionaries
merged_results_list = [{'queue': q, 'result': result} for q, result in merged_results.items()]

# Save the merged results to a new JSON file
with open('merged_results.json', 'w') as f:
    json.dump(merged_results_list, f, indent=4)
with open('grid_feasible.json', 'w') as f:
    json.dump(gridFeasible, f, indent=4)

In [ ]:
gridFeasible1 = [None]*len(reduced_compositions)
queue = startingNodes.copy()
explored = set()
calcCount = 0

In [ ]:
import logging


# Set up logging
logging.basicConfig(level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')

def apply_scheil_callable(args):
    sc, elP = args
    try:
        return sc(elP)
    except Exception as e:
        logging.error(f"Error processing item: {str(e)}")
        return None


In [ ]:


# Initialize a list to store the results
results_list = []

while len(queue) > 0:
    print(f"Queue: {queue}")
    # Calculate feasibilities of the current queue
    elPositions = [reduced_compositions[i] for i in queue]
    print(elPositions)
    # Create a list of scheil callables for each composition
    scheil_callables_list = [get_scheil_callable(system_comps_with_id[i], sc_id_to_callable, scheil_callables) for i in queue]
    
    if len(queue) > 3:
        results = process_map(apply_scheil_callable, zip(scheil_callables_list, elPositions), max_workers=4)
    else:
        results = [apply_scheil_callable((sc, elP)) for sc, elP in zip(scheil_callables_list, elPositions)]
    
    # Filter out None values (errors) and process valid results
    valid_results = [r for r in results if r is not None]
    
    # Extract only the 'Phases' component from the valid results
    phases = [result['finalPhase'] for result in valid_results]
    
    feasibilities = [set(p).issubset(set(['FCC_A1', 'BCC_A2', 'HCP_A3', 'B2_BCC', 'A2_FCC', 'L12_FCC', 'BCC2', 'A1', 'A2', 'A3', 'FCC4'])) and p != [] for p in phases]

    calcCount += len(feasibilities)
    explored = explored.union(queue)

    # Save the current step result and elPositions
    results_list.append({
        'queue': queue,
        'elPositions': elPositions,
        'results': valid_results
    })

    
    # Create next queue based on neighbors of feasible points
    nextQueue = set()
    #nextQueuePlusEquivalent = set()
    for f, i in zip(feasibilities, queue):
        # Explored point
        gridFeasible[i] = f

        # # And equivalent explored points based on system stitching
        # explored = explored.union(graphNS[i])
        # for eq in graphNS[i]:
        #     gridFeasible[eq] = f

        # Expand to neighbors of the point and equivalent points (only if the node has been feasible)
        if f:
            # Node neighbors in the same subsystem
            for n in graphN[i]:
                if n not in explored:# and n not in nextQueuePlusEquivalent:
                    nextQueue.add(n)
                    #nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])
            # Equivalent nodes neighbors in other subsystems
            for eq in graphNS[i]:
                #for n in graphN[eq]:
                if eq not in explored:# and n not in nextQueuePlusEquivalent:
                    nextQueue.add(eq)
                    #nextQueuePlusEquivalent = nextQueuePlusEquivalent.union([n] + graphNS[n])

    print(f"Calculations done: {calcCount:<5} | Explored points: {len(explored):<5}")
    queue = list(nextQueue)

# Write the results to a JSON file
with open('scheil_results.json', 'w') as f:
    json.dump(results_list, f, indent=4)


In [ ]:

# Load the results from the JSON file
with open('/ocean/projects/dmr190011p/arichte1/github_repo/AMMap/scheil_results.json', 'r') as f:
    data = json.load(f)

# Initialize a dictionary to store the merged results
merged_results = {}

# Iterate through each step in the data
for step in data:
    queue = step['queue']
    results = step['results']
    
    # Merge the queue numbers with their associated result values
    for q, result in zip(queue, results):
        merged_results[q] = result

# Convert the merged results dictionary to a list of dictionaries
merged_results_list = [{'queue': q, 'result': result} for q, result in merged_results.items()]

# Save the merged results to a new JSON file
with open('merged_scheil_results.json', 'w') as f:
    json.dump(merged_results_list, f, indent=4)
with open('grid_feasible1.json', 'w') as f:
    json.dump(gridFeasible1, f, indent=4)

In [ ]:
from pathfinding.core.graph import Graph
from pathfinding.finder.dijkstra import DijkstraFinder

In [ ]:
import json
import numpy as np
with open('grid_feasible.json', 'r') as f:
    gridFeasible = json.load(f)

In [ ]:
with open('edges.json', 'w') as file:
    json.dump(edges, file)
with open('graphNS.json', 'w') as file:
    json.dump(graphNS, file)
with open('graphN.json', 'w') as file:
    json.dump(graphN, file)

In [ ]:
pathFindEdges = []
for edge in edges:
    if gridFeasible[edge[0]] and gridFeasible[edge[1]]:
        pathFindEdges.append([edge[0], edge[1], 1])

In [ ]:
pathfindingGraph = Graph(edges=pathFindEdges, bi_directional=False)
finder = DijkstraFinder()

In [ ]:
#YAML file inputs needed here
startingComposition=[0.18, 0.74, 0.08, 0.0, 0.0]
endingComposition=[0.0, 0.0, 0.0, 0.95, 0.05]
gridElArray = np.array(compositions)
# Find the position of startingComposition
startingCompositionArray = np.array(startingComposition)
endingCompositionArray = np.array(endingComposition)
print(f"Looking for: {startingCompositionArray}")
print(f"Looking for: {endingCompositionArray}")
# New code, looking for the closest match
startingCompositionPosition = np.argmin(np.sum(np.abs(gridElArray - startingCompositionArray), axis=1))
endingCompositionPosition = np.argmin(np.sum(np.abs(gridElArray - endingCompositionArray), axis=1))
print(f"\nNode: {startingCompositionPosition} --> {gridElArray[startingCompositionPosition]}")
print(f"Node: {endingCompositionPosition} --> {gridElArray[endingCompositionPosition]}")

In [ ]:
# start_node_id = 194
# end_node_id = 830
start_node_id = startingCompositionPosition#12
end_node_id = endingCompositionPosition

# Check if the nodes exist in the graph
if start_node_id in pathfindingGraph.nodes and end_node_id in pathfindingGraph.nodes:
    path, runs = finder.find_path(
        pathfindingGraph.node(start_node_id), 
        pathfindingGraph.node(end_node_id), 
        pathfindingGraph)
    print("Path found:", path)
else:
    print(f"One or both of the nodes {start_node_id} and {end_node_id} do not exist in the graph.")

In [ ]:
shortestPath = [p.node_id for p in path]

In [ ]:
# Hover approximate formula for each point
formulas = []
for i, comp in enumerate(compositions):
    formulas.append(f"({i:>3}) "+"".join([f"{el}{100*v:.1f} " if v>0 else "" for el, v in zip(elementalSpaceComponents, comp)]))

In [ ]:
for step, i in enumerate(shortestPath):
    print(f"{step+1:>2}: {formulas[i]}")   

In [ ]:
with open('shortestPath_eq.json', 'w') as file:
    json.dump(shortestPath, file)